# Prototype for PythonQT window

date = Oktober 04, 2020 <br>
author = c.magg <br>

In [1]:
import os
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt

In [2]:
sys.path.append(os.path.abspath('../../'))

In [3]:
import vtk
from vtk.qt.QVTKRenderWindowInteractor import QVTKRenderWindowInteractor

In [4]:
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QToolBar 
from PyQt5.QtWidgets import QStatusBar, QCheckBox, QAction
from PyQt5.QtWidgets import QFrame, QVBoxLayout, QWidget
from PyQt5.QtWidgets import QPushButton, QFileDialog, QSlider
from PyQt5.QtCore import Qt

In [5]:
from slice_text import SliceText

In [6]:
path_dir = "../../Data/1/"
time_steps = [x for x in os.listdir(path_dir) if 'CT' in x or 'T1' in x]

In [7]:
class HelpingWindow(QMainWindow):
    
    def __init__(self, *args, **kwargs):
        super(HelpingWindow, self).__init__(*args, **kwargs)
        self.setWindowTitle("Navigation")
        
        widget = QWidget(self)
        self.setCentralWidget(widget)
        layout = QVBoxLayout(widget)
        
        info = QLabel(
            "left mouse button - change window level\n"+
            "right left mouse - positional zoom\n"+
            "mouse wheele - slice through volume\n")
        info.setAlignment(Qt.AlignLeft | Qt.AlignVCenter)
        layout.addWidget(info)
        
        #quit = QAction("Quit",self) 
        #quit.triggered.connect(self.closeWindow)
        #layout.addAction(quit)
        
    def closeWindow(self, s):
        self.close()

In [8]:
class VTKPipeline:
    
    def __init__(self):        
        
        self.window = None
        
        # Read data
        self.reader = vtk.vtkDICOMImageReader()
        self.UpdateReader("../../Data/1/CT/")
        
        self.dicom = vtk.vtkImageSlice()
        self.dicom.SetMapper(vtk.vtkImageSliceMapper())
        self.dicom.GetMapper().SetSliceNumber(10)
        self.dicom.GetMapper().SetInputConnection(self.reader.GetOutputPort())
               
        # Slice status message
        slice_number = self.dicom.GetMapper().GetSliceNumber()
        slice_number_max = self.dicom.GetMapper().GetSliceNumberMaxValue()
        self.sliceText = SliceText(slice_number, slice_number_max)
        
        # create and add renderer
        self.renderer = vtk.vtkRenderer() 
        self.renderer.SetBackground(0,0,0)
        self.renderer.ResetCamera()         
        self.renderer.AddViewProp(self.dicom)
        self.renderer.AddActor2D(self.sliceText.sliceTextActor)
        
        # Create interactor (customized)
        self.interactorStyle = vtk.vtkInteractorStyleImage()
        self.interactorStyle.SetInteractionModeToImageSlicing()
        # Create interactor
        self.interactor = vtk.vtkRenderWindowInteractor()
        self.interactor.SetInteractorStyle(self.interactorStyle)
        self.interactorStyle.AddObserver("MouseWheelForwardEvent", self.MoveSliceFoward)
        self.interactorStyle.AddObserver("MouseWheelBackwardEvent", self.MoveSliceBackward)
                  
    def UpdateReader(self, new_path):
        self.reader.SetDirectoryName(new_path)
        self.reader.Update()
        
    def SetWindow(self, window):
        self.window = window
    
    def MoveSliceFoward(self, obj, event):
        current_slice = self.dicom.GetMapper().GetSliceNumber()
        self.MoveSlice(current_slice + 1)

    def MoveSliceBackward(self, obj, event):
        current_slice = self.dicom.GetMapper().GetSliceNumber()
        self.MoveSlice(current_slice - 1)

    def MoveSlice(self, new_slice):
        new_slice = max(min(self.dicom.GetMapper().GetSliceNumberMaxValue(), new_slice),
                        self.dicom.GetMapper().GetSliceNumberMinValue())
        print('setting new slice', new_slice)
        self.dicom.GetMapper().SetSliceNumber(new_slice)
        slice_number = self.dicom.GetMapper().GetSliceNumber()
        self.sliceText.SetInput(new_slice, self.dicom.GetMapper().GetSliceNumberMaxValue())
        self.window.Render()

In [9]:
class VTKPipeline2:
    
    def __init__(self):
        
        # Read data
        self.path_dir = "../../Data/1/"
        self.reader = vtk.vtkDICOMImageReader()
        self.UpdateReader("../../Data/1/CT/")
        
        # Create image viewer
        self.viewer = vtk.vtkImageViewer2()
        self.viewer.SetInputData(self.reader.GetOutput())
        self.viewer.SetSlice(100)
        self.viewer.SetColorWindow(0)
        self.viewer.SetColorLevel(0)
        self.colorWindow = self.viewer.GetColorWindow()
        self.colorLevel = self.viewer.GetColorLevel()
        
    def GetInteractor(self):
        pass
        
    def UpdateReader(self, new_path):
        self.reader.SetDirectoryName(new_path)
        self.reader.Update()
        
    def GetRenderer(self):
        return self.viewer

In [10]:
class MainWindow(QMainWindow):

    def __init__(self, switch = True, parent = None):
        QMainWindow.__init__(self, parent)
        
        init_time_step = 0
        self.switch = switch
              
        # Set up frame
        self.frame = QFrame()
        self.layout = QVBoxLayout()
                              
         # VTK widget
        self.vtkWidget = QVTKRenderWindowInteractor(self.frame)
        self.layout.addWidget(self.vtkWidget)
        
        self.path_dir = "../../Data/1/"
        self.vtkPipeline = VTKPipeline()
        self.changeReader(nr_time_step=0)
        self.vtkPipeline.SetWindow(self.vtkWidget)
        self.renderer = self.vtkPipeline.renderer
        self.vtkWidget.GetRenderWindow().AddRenderer(self.renderer)
        self.interactor = self.vtkWidget.GetRenderWindow().GetInteractor() 
        self.interactor.SetInteractorStyle(self.vtkPipeline.interactorStyle)
        
        self.frame.setLayout(self.layout)
        self.setCentralWidget(self.frame)
        
        # Toolbar with Slider
        self.toolbar = QToolBar("Time slider") 
        self.addToolBar(self.toolbar)
        self.createSliderToolbar()        
        
        # Menu Bar
        bar = self.menuBar()        
        new_folder = QAction("New Patient",self)
        new_folder.triggered.connect(self.openFolder)
        #new_folder.setShortcut("Ctrl+N")
        bar.addAction(new_folder)
              
        helping = QAction("Help", self)
        helping.triggered.connect(self.helping_button)
        bar.addAction(helping)
        
        quit = QAction("Quit",self) 
        quit.triggered.connect(self.closeWindow)
        bar.addAction(quit)
        
        # Show and initialize
        self.show()
        self.vtkWidget.Initialize()
        
    def closeWindow(self, s):
        self.close()
        
    def createSliderToolbar(self, init_value=0):
        self.removeToolBar(self.toolbar)
        self.toolbar = QToolBar("Time slider") 
        
        self.slider = QSlider(Qt.Horizontal)
        self.slider.valueChanged.connect(self.timeStepChange)
        self.slider.setMinimum(0)
        self.slider.setMaximum(len(self.time_steps)-1)
        if init_value > len(self.time_steps)-1:
            init_value = len(self.time_steps)-1
        self.slider.setValue(init_value)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(1)    
        print('Create slider {0}/{1}'.format(init_value, len(self.time_steps)-1))
        self.toolbar.addWidget(self.slider)
        
        self.toolbar.addSeparator()
        
        background_toggle = QCheckBox()
        background_toggle.setText("Background Toggle")
        background_toggle.setChecked(True)
        background_toggle.stateChanged.connect(self.backgroundToogleChange)
        
        self.toolbar.addWidget(background_toggle)
        self.addToolBar(self.toolbar)
        
    def backgroundToogleChange(self, s):
        pass
        if s == 0:
            print('no background')
            self.colorWindow = self.viewer.GetColorWindow()
            self.colorLevel = self.viewer.GetColorLevel()
            self.viewer.SetColorLevel(255)
            self.viewer.SetColorWindow(0)
            self.viewer.Render()
        elif s == 2:
            print('background')
            self.viewer.SetColorLevel(self.colorLevel)
            self.viewer.SetColorWindow(self.colorWindow)
            self.viewer.Render()
        
    def openFolder(self, s):        
        file_dialog = QFileDialog()
        file_dialog.setFileMode(QFileDialog.DirectoryOnly)
        file_dialog.setViewMode(QFileDialog.Detail)
        
        if file_dialog.exec_():
            self.path_dir = file_dialog.selectedFiles()[0]
            print('Open folder', self.path_dir)
            self.changeReader()
            self.createSliderToolbar(init_value=self.slider.value())
            
    def changeReader(self, nr_time_step=None):
        self.time_steps = [x for x in os.listdir(self.path_dir) if 'CT' in x or 'T1' in x]
        if nr_time_step is None:
            nr_time_step = min(len(self.time_steps)-1,self.slider.value())
            self.slider.setValue(nr_time_step)
        print(nr_time_step)
        path_dicom_dir = os.path.join(self.path_dir, self.time_steps[nr_time_step]) 
        self.vtkPipeline.UpdateReader(path_dicom_dir)
        print('Change folder', path_dicom_dir)
        
    def helping_button(self):
        self.helpingWindow = HelpingWindow()
        self.helpingWindow.show()
            
    def timeStepChange(self,s):
        print('Change step {0}/{1}'.format(self.slider.value(),len(self.time_steps)-1))
        self.changeReader()       

In [11]:
app = QApplication(sys.argv)
window = MainWindow(True)
window.show()

app.exec_()

0
Change folder ../../Data/1/CT
Create slider 0/3
setting new slice 11
setting new slice 12
setting new slice 13
setting new slice 14
setting new slice 15
setting new slice 16
setting new slice 17
setting new slice 18
setting new slice 19
setting new slice 20
setting new slice 21
setting new slice 22
setting new slice 23
setting new slice 24
setting new slice 25
setting new slice 26
setting new slice 27
setting new slice 28
setting new slice 29
setting new slice 30
setting new slice 31
setting new slice 32
setting new slice 33
setting new slice 34
setting new slice 35
setting new slice 36
setting new slice 37
setting new slice 38
setting new slice 39
setting new slice 40
setting new slice 41
setting new slice 42
setting new slice 43
setting new slice 44
setting new slice 45
setting new slice 46
setting new slice 47
setting new slice 48
setting new slice 49
setting new slice 50
setting new slice 51
setting new slice 52
setting new slice 53
setting new slice 54
setting new slice 55
setti

0

In [12]:
window.vtkWidget.GetRenderWindow().GetInteractor()

(vtkRenderingCorePython.vtkGenericRenderWindowInteractor)000000CFDCCC91C8

In [13]:
window.vtkWidget.Render

<bound method QVTKRenderWindowInteractor.Render of <vtk.qt.QVTKRenderWindowInteractor.QVTKRenderWindowInteractor object at 0x000000CFDCF1E168>>